In [2]:
from glob import glob
import os
import IPython.display as display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import datetime
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from IPython.display import clear_output


In [10]:
# important for reproducibility
SEED = 42

# https://www.tensorflow.org/guide/data_performance#prefetching
AUTOTUNE = tf.data.experimental.AUTOTUNE

base_dir = os.path.abspath('..')
train_dir = os.path.join(base_dir,"data","train")
val_dir = os.path.join(base_dir,"data","test")

In [13]:
# Image size that we are going to use
IMG_SIZE = 128
# Our images are RGB (3 channels)
N_CHANNELS = 3
# Number of classes + 1 for background
N_CLASSES = 4

In [23]:
def parse_image(img_path: str) -> dict:
    
    """Load an image and its annotation (mask) and returning
    a dictionary.

    Parameters
    ----------
    img_path : str
        Image (not the mask) location.

    Returns
    -------
    dict
        Dictionary mapping an image and its ground truth
    """
    image = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.uint8)


    mask_path = tf.strings.regex_replace(img_path, "images", "masks_new")
    mask_path = tf.strings.regex_replace(mask_path, "jpg", "tif")
    
    mask = tf.io.read_file(mask_path)
    
    # The masks contain a class index for each pixels
    mask = tf.image.decode_tiff(image)(mask, channels=1)
   

    return {'image': image, 'segmentation_mask': mask}

In [24]:
train_dataset = tf.data.Dataset.list_files(os.path.join(train_dir , 'original_retinal_images' , "*.jpg"), seed=SEED)
train_dataset = train_dataset.map(parse_image)


AttributeError: in user code:

    <ipython-input-23-d5ee3c7bb398>:27 parse_image  *
        mask = tf.image.decode_tiff(image)(mask, channels=1)

    AttributeError: module 'tensorflow._api.v2.image' has no attribute 'decode_tiff'


In [25]:
import tensorflow_io

ModuleNotFoundError: No module named 'tensorflow_io'

In [26]:
!pip install tensorflow-io